# Final Project

The world online is filled with negativity nowadays, from the comments people leave to each other online, to news that renowned agencies publish. 

It is very important to determine the positive news and negative news from each other. This lab tries to tackle this problem by using naive bayes to classify news as positive, or negative.



## Required Files and Libraries:
### Files:

The files below provide the diferent stopwords and as well as they're values.

<a href="https://drive.google.com/file/d/1OWM3xqn35Hdf04r7QwKSDbbbJ45jQaCN/view
">Label 1</a>

<a href="https://drive.google.com/file/d/14JDy3NsflKCst_4nunwsC4Ehs154Otnr/view
">Label 2</a>

### Libraries:

- bs4 is (only) used for reading news.
<!-- - the rest are commonly used libraries. -->

## 1.1 Import required libraries

In [1]:
import urllib
import urllib.request
import os
import re
import sys
import string
import unicodedata
import getopt
from xml.dom import minidom

from bs4 import BeautifulSoup

import math


## 1.2 Read News


"urls" should be a list containing links to news.

"include_urls" should be a boolean value.

"num" should be an integer value stating how many news article one wants.

The return value is a dictionary with the key as the url to the specific news site, and value as the actual news itself, stored as a string

In [2]:
def read_news(urls,include_urls,num):
    searcher = re.compile(r'([a-z]+) ([a-z]+)', re.IGNORECASE)
    count = 0
    result={}
    for url in urls:
        feed = urllib.request.urlopen(url)
        print(feed)
        try:
            dom = minidom.parse(feed)
            forecasts = []
            for node in dom.getElementsByTagName('title'):
                txt = node.firstChild.wholeText
#                 print("1")
                if searcher.search(txt):
#                     print("2")
                    
                    txt = unicodedata.normalize('NFKD', txt).encode('ascii', 'ignore')
                    print(txt)
                    if include_urls:
#                         print("3")
                        p = node.parentNode
                        link = p.getElementsByTagName('link')[0].firstChild.wholeText
#                         print(link)

                        #read
                        page=urllib.request.urlopen(link)
                        contents=page.read()
                        soup=BeautifulSoup(contents,"html.parser")
                        text=[]
                        for tag in soup.find_all('p', class_='font--body font-copy gray-darkest ma-0 pb-md'):
                            text.append(tag.get_text().strip())
                        result[link]=text
                        count += 1
                        if count >=num:
#                             print(count)
                            return result
                            sys.exit(0)
        except:
            
            return result
            sys.exit(1)
    return result
    sys.exit(0)


In [3]:
links = ["http://feeds.washingtonpost.com/rss/rss_election-2012"]
         
include = True

read_result=read_news(links,include,4)


Below is one (key, value) pair, with value taking the first 500 letters so as to not take too much space.

In [8]:
for i in read_result.keys():
    print("Key: \n\n",i)
    print("\nValue: \n \n",read_result[i][0][:500]) #list at 0, first 500 letters
    break

Key: 

 https://www.washingtonpost.com/politics/electoral-college-affirms-bidens-victory-on-a-relatively-calm-day-of-a-chaotic-election/2020/12/14/0994b232-3e48-11eb-9453-fc36ba051781_story.html?

Value: 
 
 President-elect Joe Biden achieved formal victory over President Trump on Monday, winning his 306 votes in the electoral college and advancing one more step toward inauguration even as die-hard Trump supporters redoubled their efforts to stop the normal transfer of power.Electors gathered in every state and the District of Columbia for a day-long series of votes that delivered no surprises for either Trump or Biden. The proceedings harked back to more typical presidential elections and stood in 


## 1.2 Process news

### 1.2.1 Process news using stanford sentiment treebank

In [9]:
ci_score= {}
negative_word={}
positive_word={}
stop_word=[]
stop_word.append("")

In [10]:
with open('./stanfordSentimentTreebank/dictionary.txt')as f:
    ci={}
    for line in f:
        line=line.strip().split('|')
        ci[line[0]]=int(line[1])
        

In [11]:
score=open('./stanfordSentimentTreebank/sentiment_labels.txt')
score_read=score.readline()
score_read=score.readline().strip().split("|")
while score_read[0]!='':
    ci_score[int(score_read[0])]=float(score_read[1])
    score_read=score.readline().strip().split("|")


In [12]:
for key in ci.keys():
    if ci_score[ci[key]]<=0.2:
        negative_word[key.lower()]=1-ci_score[ci[key]]
    if ci_score[ci[key]]<=0.4 and ci_score[ci[key]]>0.2:
        negative_word[key.lower()] = 1 - ci_score[ci[key]]
    if ci_score[ci[key]]<=0.6 and ci_score[ci[key]]>0.4:
        stop_word.append(key.lower())
    if ci_score[ci[key]]<=0.8 and ci_score[ci[key]]>0.6:
        positive_word[key.lower()] = ci_score[ci[key]]
    if ci_score[ci[key]]<=1 and ci_score[ci[key]]>0.8:
        positive_word[key.lower()] = ci_score[ci[key]]

In [13]:
for key in negative_word.keys():
    negative_word[key]=negative_word[key]*(1.0/len(negative_word))*(len(negative_word)/(len(negative_word) + len(positive_word)))
for key in positive_word.keys():
    positive_word[key]=positive_word[key]*(1.0/len(positive_word))*(len(positive_word)/(len(negative_word) + len(positive_word)))


one (key, value) pair of negative_word, as well as one (key,value) pair of positive_word

In [14]:
for i in negative_word.keys():
    print("Negative Key: \n\n",i)
    print("\nNegative Value: \n \n",negative_word[i]) 
    break
    

print()
print("--------")
print("--------")
print()

for i in positive_word.keys():
    print("Positive Key: \n\n",i)
    print("\nPositive Value: \n \n",positive_word[i]) 
    break


Negative Key: 

 ! oh , look at that clever angle ! wow , a jump cut !

Negative Value: 
 
 6.108963568848701e-06

--------
--------

Positive Key: 

 ! brilliant

Positive Value: 
 
 7.283777268382633e-06


### 1.2.2 Process News using moody_news

In [15]:
positive_dic={}
negative_dic={}
stop_word=[]


In [16]:
with open('positive-words.txt')as f:
    for line in f:
        line=line.strip()
        positive_dic[line]=0
with open('negative-words.txt')as f:
    for line in f:
        line=line.strip()
        negative_dic[line]=0
with open('stopwords.txt')as f:
    for line in f:
        stop_word.append(line)
stop_word.append("")

In [17]:
for positive in positive_dic.keys():
    positive_dic[positive]=1.0/(len(positive_dic))*len(positive_dic)/(len(positive_dic)+len(negative_dic))
for negative in negative_dic.keys():
    negative_dic[negative]=1.0/(len(negative_dic))*len(negative_dic)/(len(positive_dic)+len(negative_dic))

Again, below is a sample (key, value) pair of negative_dic and positive_dic.

In [18]:
for i in negative_dic.keys():
    print("Negative Key: \n\n",i)
    print("\nNegative Value: \n \n",negative_dic[i]) 
    break
    

print()
print("--------")
print("--------")
print()

for i in positive_dic.keys():
    print("Positive Key: \n\n",i)
    print("\nPositive Value: \n \n",positive_dic[i]) 
    break


Negative Key: 

 2-faced

Negative Value: 
 
 0.00014729709824716454

--------
--------

Positive Key: 

 abound

Positive Value: 
 
 0.0001472970982471645


## 1.3 Naive Bayes Algorithm

The bayes below takes in a text, a positive dictionary and a negative dictionary, and a calP

The "text" should be a dictionary, with key as the link, and value as the article itself.

Positive and negative dictionaries should be the dictionaries above.

calP will be specified, and will have small changes depending on the type of positive/negative dictionary used.

In [19]:
def bayes(text,pos,neg,cal):
    pGood = cal(text,pos)
    pBad = cal(text,neg)
    print("Good: ",pGood)
    print("Bad: ",pBad)
    if pGood >= pBad:
        print("positive")
    else:
        print("negative")

### 1.3.1 Stanford Sentiment Treebank

calPSt takes in the result from above, as well as one dictionary.

The dictionary can be the positive dictionary or the negative dictionary.


In [20]:
def calPSt(word_list,traning_dic):
    p_list = []
    for word in word_list:
        word = word.strip().lower()
        if word in traning_dic:
            p_list.append(str(traning_dic[word]))
        elif len(word.split(' '))>1:
            pattern = r',|\.|/|;|\'|`|\[|\]|<|>|\?|:|"|\{|\}|\~|!|@|#|\$|%|\^|&|\(|\)|-|=|\_|\+|，|。|、|；|‘|’|【|】|·|！| |…|（|）'
            result_list = re.split(pattern, word)
            word_list_nostop = []
            for j in result_list:
                if j in stop_word:
                    pass
                else:
                    word_list_nostop.append(j)
#             print(word_list_nostop)
            for i in word_list_nostop:
                if i in traning_dic:
                    p_list.append(str(traning_dic[i]))
        else:
            p_list.append(str(0.5*(1.0+len(traning_dic) / (len(negative_word) + len(positive_word))+2)))
    # 计算P
    p_index = 0
    for p in p_list:
        p = math.log(float(p), 2)
        p_index = p_index + p
    return -p_index

In [21]:
for i in read_result.keys():
    print(i)
    bayes(read_result[i],positive_word,negative_word,calPSt)

https://www.washingtonpost.com/politics/electoral-college-affirms-bidens-victory-on-a-relatively-calm-day-of-a-chaotic-election/2020/12/14/0994b232-3e48-11eb-9453-fc36ba051781_story.html?
Good:  1851.1048044417578
Bad:  957.1979672016964
positive
https://www.washingtonpost.com/politics/2020/12/16/joe-biden-trump-transition-live-updates/
Good:  1606.1498147227567
Bad:  576.0873684681163
positive
 https://www.washingtonpost.com/politics/electoral-college-affirms-bidens-victory-on-a-relatively-calm-day-of-a-chaotic-election/2020/12/14/0994b232-3e48-11eb-9453-fc36ba051781_story.html?utm_source=rss&utm_medium=referral&utm_campaign=wp_politics
Good:  1956.3014890895881
Bad:  992.3212616336838
positive
https://www.washingtonpost.com/politics/biden-mcconnell-senate-georgia/2020/12/16/72919d6a-3e5a-11eb-9453-fc36ba051781_story.html?utm_source=rss&utm_medium=referral&utm_campaign=wp_politics
Good:  1413.762833585449
Bad:  748.9360236424741
positive


### 1.3.2 Moody_news

In [22]:
def calPMood(word_list,traning_dic):
    p_list = []
    for word in word_list:
        word = word.strip().lower()
        pattern = r',|\.|/|;|\'|`|\[|\]|<|>|\?|:|"|\{|\}|\~|!|@|#|\$|%|\^|&|\(|\)|-|=|\_|\+|，|。|、|；|‘|’|【|】|·|！| |…|（|）'
        result_list = re.split(pattern, word)
        word_list_nostop = []
        for j in result_list:
            if j in stop_word:
                pass
            else:
                word_list_nostop.append(j)
        # print(word_list_nostop)
        for text_word in word_list_nostop:
            if text_word in traning_dic:
                    p_list.append(str(traning_dic[text_word]))
            else:
                p_list.append(str(1.0+len(traning_dic) / (len(positive_dic) + len(negative_dic))+1))
    # 计算P
    p_index = 0
    for p in p_list:
        p = math.log(float(p), 2)
        p_index = p_index + p
    return -p_index

In [23]:
for i in read_result.keys():
    print(i)
    bayes(read_result[i],positive_word,negative_word,calPMood)



https://www.washingtonpost.com/politics/electoral-college-affirms-bidens-victory-on-a-relatively-calm-day-of-a-chaotic-election/2020/12/14/0994b232-3e48-11eb-9453-fc36ba051781_story.html?
Good:  -4401.6438426119275
Bad:  -5080.2259042719725
positive
https://www.washingtonpost.com/politics/2020/12/16/joe-biden-trump-transition-live-updates/
Good:  -3620.92247590893
Bad:  -4525.122280744154
positive
 https://www.washingtonpost.com/politics/electoral-college-affirms-bidens-victory-on-a-relatively-calm-day-of-a-chaotic-election/2020/12/14/0994b232-3e48-11eb-9453-fc36ba051781_story.html?utm_source=rss&utm_medium=referral&utm_campaign=wp_politics
Good:  -4659.487242883224
Bad:  -5399.07759846886
positive
https://www.washingtonpost.com/politics/biden-mcconnell-senate-georgia/2020/12/16/72919d6a-3e5a-11eb-9453-fc36ba051781_story.html?utm_source=rss&utm_medium=referral&utm_campaign=wp_politics
Good:  -4204.546830114422
Bad:  -4650.0095973136995
positive


As these are news, I paid the $1 to read the news, account available from December 10 to around January 10 

username: jhan18@simons-rock.edu

password: ILikePotatoes

In my opinion:

1. https://www.washingtonpost.com/politics/electoral-college-affirms-bidens-victory-on-a-relatively-calm-day-of-a-chaotic-election/2020/12/14/0994b232-3e48-11eb-9453-fc36ba051781_story.html?utm_source=rss&utm_medium=referral&utm_campaign=wp_politics 

is positive news

2. https://www.washingtonpost.com/politics/2020/12/16/joe-biden-trump-transition-live-updates/?utm_source=rss&utm_medium=referral&utm_campaign=wp_politics

I feel like this is positive news as well, as it mentions how Binden is trying to rebuild the nation.

3. https://www.washingtonpost.com/politics/biden-mcconnell-senate-georgia/2020/12/16/72919d6a-3e5a-11eb-9453-fc36ba051781_story.html?utm_source=rss&utm_medium=referral&utm_campaign=wp_politics

I feel that this is a positive article, it did not say how they loved Biden, but they gave many voices that people loved Biden. So it is implying that Biden is a good president.

4. https://www.washingtonpost.com/local/obituaries/james-flug-who-helped-block-nixon-nominees-and-investigated-watergate-dies-at-81/2020/12/15/a14a59e0-3eea-11eb-8bc0-ae155bee4aff_story.html?utm_source=rss&utm_medium=referral&utm_campaign=wp_politics

I think that the news itself is a negative article, it is saying that a really famous and loved person died at the age of 81.

But at the same time, I understand how the NB algorithm classified it as positive, because inside the article, it contained a lot of words of people loving him and praising him.

